## AHG RECCOMENDATION ENGINE

In [47]:
# import libraries 

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import pyodbc   # # Connection to SQL server
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import pyodbc
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split # for engine evaluation
from sklearn.metrics import mean_squared_error # for engine evaluation
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split # for engine evaluation
from sklearn.metrics import mean_squared_error # for engine evaluation

In [ ]:
## Function

In [48]:
## Connecting to SQL server database to bring out the sql table:

def fetch_sql_table(table_name):
    
    conn = pyodbc.connect("DRIVER={SQL SERVER};"
                       "SERVER=DESKTOP-PMHMTM1;"
                        "DATABASE=AdventureWorksDW2022;"
                          "Trusted_Connection=yes"
                     )

    # Get our Table

    sql_for_df = f'Select FIS.[OrderQuantity] as QuantityOrdered, psc.[EnglishProductSubcategoryName] as ProductSub,FIS.[CustomerKey] as CustomerID, p.[EnglishProductName] as Product,PC.[EnglishProductCategoryName] as ProductCategory from [dbo].[FactInternetSales] as FIS INNER JOIN [dbo].[DimProduct] as P on FIS.[ProductKey] = p.[ProductKey] left join [dbo].[DimProductSubcategory] as psc on psc.[ProductSubcategoryKey] = p.[ProductSubcategoryKey] left join [dbo].[DimProductCategory] as PC on PC.[ProductCategoryKey] = psc.[ProductCategoryKey]' 

     #two parameters: 1. the dataset to extract
     #                2. the connection we created
    df = pd.read_sql(sql_for_df, conn)
    conn.close()
    
    
    return df

 # This function checks each column of a DataFrame for missing values and returns a report indicating the presence and count of any missing records.

def missingValFinder(dataset):
    
    MissingReprt = []
    df = dataset
    cols = df.columns.tolist()
    MissingRecs = df.isnull().sum()
    
    for col, rec in zip(cols, MissingRecs):
        
        if rec > 0:
            MissingReprt.append(f'{col} has {rec} missing records')
        else:
            MissingReprt.append(f'{col} has no missing records')
            
            
    return MissingReprt

In [49]:
# Using the function created above to call Factsinternetsales table

table_name = 'FactInternetSales'

FIS = fetch_sql_table(table_name)

FIS.head()

C:\Users\iife\AppData\Local\Temp\ipykernel_21344\85392130.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_for_df, conn)


,QuantityOrdered,ProductSub,CustomerID,Product,ProductCategory
0,1,Road Bikes,21768,"Road-150 Red, 62",Bikes
1,1,Mountain Bikes,28389,"Mountain-100 Silver, 44",Bikes
2,1,Mountain Bikes,25863,"Mountain-100 Silver, 44",Bikes
3,1,Road Bikes,14501,"Road-650 Black, 62",Bikes
4,1,Mountain Bikes,11003,"Mountain-100 Silver, 44",Bikes


In [114]:
# Using the function created above to call Factsinternetsales table

table_name = 'FactInternetSales'

FIS = fetch_sql_table(table_name)

FIS.tail()

C:\Users\iife\AppData\Local\Temp\ipykernel_21344\85392130.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_for_df, conn)


,QuantityOrdered,ProductSub,CustomerID,Product,ProductCategory
60393,1,Fenders,15868,Fender Set - Mountain,Accessories
60394,1,Caps,15868,AWC Logo Cap,Clothing
60395,1,Fenders,18759,Fender Set - Mountain,Accessories
60396,1,Bike Stands,18759,All-Purpose Bike Stand,Accessories
60397,1,Caps,18759,AWC Logo Cap,Clothing


In [51]:
## Overview of the FactInternetSales

FIS.shape

(60398, 5)

In [52]:
## Column Names in the FactInternetSales

FIS.columns

Index(['QuantityOrdered', 'ProductSub', 'CustomerID', 'Product',
       'ProductCategory'],
      dtype='object')

In [53]:
## A summary of the count of null and non-null values in each column can provide a quick snapshot of data completeness.
    
FIS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   QuantityOrdered  60398 non-null  int64 
 1   ProductSub       60398 non-null  object
 2   CustomerID       60398 non-null  int64 
 3   Product          60398 non-null  object
 4   ProductCategory  60398 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.3+ MB


In [54]:
## Statistical Summary of FactInternetSales Dataset

FIS.describe()

,QuantityOrdered,CustomerID
count,60398.0,60398.000000
mean,1.0,18841.685420
std,0.0,5432.430404
min,1.0,11000.000000
25%,1.0,14003.000000
50%,1.0,18143.000000
75%,1.0,23429.750000
max,1.0,29483.000000


In [55]:
def missing_val_report(df):
    # Generate report on missing values
    missing_report = pd.DataFrame({
        'Column': df.columns,
        'Missing Records': df.isnull().sum(),
        'Total Records': len(df),
        'Percentage Missing': (df.isnull().sum() / len(df)) * 100
    })
    
    # Filter to show only columns with missing records
    return missing_report[missing_report['Missing Records'] > 0]

# Assuming FIS is your DataFrame
missing_report = missing_val_report(FIS)
print(missing_report)

Empty DataFrame
Columns: [Column, Missing Records, Total Records, Percentage Missing]
Index: []


In [56]:
# outputing a DataFrame showing columns with missing values

missingValFinder(FIS)

['QuantityOrdered has no missing records',
 'ProductSub has no missing records',
 'CustomerID has no missing records',
 'Product has no missing records',
 'ProductCategory has no missing records']

In [26]:
## Get a quick overview of the distribution of each unique value within a categorical column

for column in FIS.select_dtypes(include='object').columns:
    print(f"Value counts for {column}:\n", FIS[column].value_counts(), "\n")

Value counts for ProductSub:
 ProductSub
Tires and Tubes      17332
Road Bikes            8068
Bottles and Cages     7981
Helmets               6440
Mountain Bikes        4970
Jerseys               3332
Caps                  2190
Touring Bikes         2167
Fenders               2121
Gloves                1430
Shorts                1019
Cleaners               908
Hydration Packs        733
Socks                  568
Vests                  562
Bike Racks             328
Bike Stands            249
Name: count, dtype: int64 

Value counts for Product:
 Product
Water Bottle - 30 oz.      4244
Patch Kit/8 Patches        3191
Mountain Tire Tube         3095
Road Tire Tube             2376
Sport-100 Helmet, Red      2230
                           ... 
Mountain-500 Silver, 42      45
Mountain-100 Silver, 42      42
Mountain-500 Black, 52       41
Mountain-500 Silver, 44      39
Mountain-100 Silver, 48      36
Name: count, Length: 130, dtype: int64 

Value counts for ProductCategory:
 ProductCa

In [57]:
# filtering the DataFrame, by Dropping the columns 'Product' and 'ProductCategory

df = FIS.drop(columns=['Product', 'ProductCategory'])

# Display the cleaned DataFrame
print(FIS_cleaned.head())

df.head()

   QuantityOrdered      ProductSub  CustomerID
0                1      Road Bikes       21768
1                1  Mountain Bikes       28389
2                1  Mountain Bikes       25863
3                1      Road Bikes       14501
4                1  Mountain Bikes       11003


,QuantityOrdered,ProductSub,CustomerID
0,1,Road Bikes,21768
1,1,Mountain Bikes,28389
2,1,Mountain Bikes,25863
3,1,Road Bikes,14501
4,1,Mountain Bikes,11003


In [58]:
## saving the DataFrame to a CSV file

# df.to_csv('your_file_name.csv', index=False)

In [59]:
df.shape

(60398, 3)

## User based similarity Approach

A collaborative filtering technique commonly used in recommendation systems. It recommends items to users based on the preferences of similar users. 
To build a recommendation engine, we need to create a user-item interaction matrix. 

### Step-by-Step Guide
#### 1. Prepare the Data: 
since we have a DataFrame with CustomerID, ProductCategory, and QuantityOrdered. 

#### 2. we Create the User-Item Interaction Matrix: 
we use CustomerID as rows, ProductCategory as columns, and QuantityOrdered as values.
then Convert categorical columns like CustomerID and ProductCategory to integer indices for compatibility with the sparse matrix.


#### 3. Use csr_matrix for Memory Efficiency:
Create a sparse matrix where only non-zero elements are stored, saving memory and allowing faster computation.


In [62]:
### Building a Memory-Efficient Customer-Item Interaction Matrix Using Sparse Matrices for Recommendation Systems

# Create the customer-item interaction matrix
customer_item_matrix = df.pivot_table(index='CustomerID', columns='ProductSub', values='QuantityOrdered', fill_value=0)

# Convert the matrix to a sparse matrix
customer_item_matrix_sparse = csr_matrix(customer_item_matrix.values)

In [67]:
# Outputting the customer-item interaction matrix
customer_item_matrix

ProductSub,Bike Racks,Bike Stands,Bottles and Cages,Caps,Cleaners,Fenders,Gloves,Helmets,Hydration Packs,Jerseys,Mountain Bikes,Road Bikes,Shorts,Socks,Tires and Tubes,Touring Bikes,Vests
CustomerID,,,,,,,,,,,,,,,,,
11000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
11001,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
11002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
11003,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
11004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
29480,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
29481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
customer_item_matrix.shape

(18484, 17)

In [69]:
customer_item_matrix_sparse

<18484x17 sparse matrix of type '<class 'numpy.float64'>'
	with 44734 stored elements in Compressed Sparse Row format>

### we calculate and visualize customer similarity using Cosine Similarity for Recommendations.
1. Compute the Cosine Similarity Matrix (customer_item_matrix_sparse) which computes the similarity between each pair of customers based on their interactions in the sparse matrix.
   
2. Convert to DataFrame which makes the similarity matrix easier to interpret by using the original customer IDs as row and column labels

    This similarity matrix can now be used for recommending items based on similar customers’ preferences

In [70]:
# Compute the cosine similarity matrix
customer_similarity_sparse = cosine_similarity(customer_item_matrix_sparse)

# Convert the similarity matrix back to a DataFrame for better readability
customer_similarity_df = pd.DataFrame(customer_similarity_sparse, index=customer_item_matrix.index, columns=customer_item_matrix.index)

In [72]:
# Output the similarity DataFrame to review

customer_similarity_df

CustomerID,11000,11001,11002,11003,11004,11005,11006,11007,11008,11009,...,29474,29475,29476,29477,29478,29479,29480,29481,29482,29483
CustomerID,,,,,,,,,,,,,,,,,,,,,
11000,1.000000,0.617213,0.707107,0.547723,0.816497,0.707107,0.471405,0.912871,0.730297,0.816497,...,0.408248,0.408248,0.408248,0.471405,0.288675,0.408248,0.408248,0.408248,0.408248,0.408248
11001,0.617213,1.000000,0.436436,0.507093,0.566947,0.218218,0.436436,0.507093,0.507093,0.377964,...,0.377964,0.377964,0.377964,0.436436,0.267261,0.377964,0.566947,0.377964,0.377964,0.377964
11002,0.707107,0.436436,1.000000,0.516398,0.866025,0.666667,0.666667,0.774597,0.774597,0.866025,...,0.577350,0.577350,0.577350,0.666667,0.408248,0.577350,0.577350,0.577350,0.577350,0.577350
11003,0.547723,0.507093,0.516398,1.000000,0.447214,0.774597,0.774597,0.600000,0.800000,0.670820,...,0.447214,0.447214,0.447214,0.516398,0.632456,0.447214,0.670820,0.447214,0.447214,0.447214
11004,0.816497,0.566947,0.866025,0.447214,1.000000,0.577350,0.577350,0.894427,0.670820,0.750000,...,0.500000,0.500000,0.500000,0.577350,0.353553,0.500000,0.500000,0.500000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29479,0.408248,0.377964,0.577350,0.447214,0.500000,0.577350,0.577350,0.447214,0.447214,0.500000,...,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
29480,0.408248,0.566947,0.577350,0.670820,0.500000,0.288675,0.577350,0.447214,0.670820,0.500000,...,0.000000,0.000000,0.000000,0.866025,0.707107,0.000000,1.000000,0.000000,0.000000,0.000000
29481,0.408248,0.377964,0.577350,0.447214,0.500000,0.577350,0.577350,0.447214,0.447214,0.500000,...,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000


In [73]:
# Function to Recommend Similar Customers Based on Customer Similarity Scores:


def recommend_similar_customers(CustomerID, customer_similarity_df, num_recommendations=5):
    # Get the similarity scores for all customers with the given customer id
    sim_scores = customer_similarity_df.loc[CustomerID]

    # Sort the customers based on the similarity scores
    sim_scores = sim_scores.sort_values(ascending=False)

    # Get the indices of the most similar customers
    recommended_customers = sim_scores.index[1:num_recommendations+1].tolist()

    return recommended_customers

In [ ]:
## Testing the reccomendation engine:
Generate recommendations for a specific user:

In [113]:
### Generating Customer Recommendations Based on Similarity Check and Validation

# # Specify the customer ID and number of recommendations to test the engine
CustomerID = 21784  # # Replace with an actual customer ID in the dataset.
num_recommendations = 8

# Generate and display recommendations
if CustomerID in customer_similarity_df.index:
    recommended_customers = recommend_similar_customers(CustomerID, customer_similarity_df, num_recommendations)
    print(f"Recommended customers similar to {CustomerID}: {recommended_customers}")

else:
    print(f'{CustomerID} does not exist, please enter a valid customer ID')

Recommended customers similar to 21784: [22957, 17554, 24869, 23650, 16994, 11596, 24859, 17560]


### Item based similarity approach

To build an item-based recommendation engine using the item-customer interaction matrix, we can create a matrix where each row represents a ProductSubcategory, each column represents a CustomerID, and the values represent QuantityOrdered. This matrix will be converted to a sparse format using csr_matrix from scipy.sparse to optimize memory usage.

Step-by-Step Guide
Prepare the Data:

Make sure you have a DataFrame with columns ProductSubcategory, CustomerID, and QuantityOrdered.
Create the Item-Customer Interaction Matrix:

Use ProductSubcategory as rows, CustomerID as columns, and QuantityOrdered as values.
Fill missing values with 0 to represent cases where there is no interaction.
Convert to Sparse Matrix for Efficiency:

Use csr_matrix to store the interaction matrix as a sparse matrix.

In [84]:
## Creating and Optimizing the Item-Customer Interaction Matrix for Item-Based Recommendations

# Create the item-user interaction matrix
item_customer_matrix = df.pivot_table(index='ProductSub', columns='CustomerID', values='QuantityOrdered', fill_value=0)

# Convert to a sparse matrix
item_customer_matrix_sparse = csr_matrix(item_customer_matrix.values)

# Display the Interaction Matrix:
item_customer_matrix

CustomerID,11000,11001,11002,11003,11004,11005,11006,11007,11008,11009,...,29474,29475,29476,29477,29478,29479,29480,29481,29482,29483
ProductSub,,,,,,,,,,,,,,,,,,,,,
Bike Racks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bike Stands,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottles and Cages,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
Caps,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
Cleaners,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fenders,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Gloves,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Helmets,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
Hydration Packs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
item_customer_matrix.shape

(17, 18484)

In [83]:
## Calculating Item Similarity Using Cosine Similarity for Item-Based Recommendations

# Step 1: Compute the cosine similarity matrix from the sparse item-customer matrix
item_similarity_sparse = cosine_similarity(item_customer_matrix_sparse)

# Step 2: Convert the similarity matrix to a DataFrame for readability
item_similarity_df = pd.DataFrame(item_similarity_sparse, index=item_customer_matrix.index, columns=item_customer_matrix.index)

# Display the item similarity DataFrame
item_similarity_df.head()

ProductSub,Bike Racks,Bike Stands,Bottles and Cages,Caps,Cleaners,Fenders,Gloves,Helmets,Hydration Packs,Jerseys,Mountain Bikes,Road Bikes,Shorts,Socks,Tires and Tubes,Touring Bikes,Vests
ProductSub,,,,,,,,,,,,,,,,,
Bike Racks,1.000000,0.024909,0.059222,0.050456,0.071259,0.084531,0.040375,0.045985,0.014481,0.043200,0.078939,0.046467,0.034754,0.025807,0.124014,0.033551,0.023503
Bike Stands,0.024909,1.000000,0.027586,0.037512,0.028193,0.050276,0.041505,0.038224,0.014354,0.038605,0.039125,0.034489,0.018086,0.008140,0.148294,0.023558,0.021745
Bottles and Cages,0.059222,0.027586,1.000000,0.239571,0.158908,0.169153,0.167492,0.279082,0.199080,0.203142,0.288934,0.294780,0.028800,0.079650,0.130675,0.249847,0.115606
Caps,0.050456,0.037512,0.239571,1.000000,0.043197,0.132486,0.050211,0.190201,0.060576,0.264116,0.173069,0.182236,0.171648,0.028396,0.178635,0.193685,0.025694
Cleaners,0.071259,0.028193,0.158908,0.043197,1.000000,0.095675,0.059238,0.054299,0.156334,0.046672,0.093047,0.090453,0.037066,0.024307,0.198123,0.064264,0.021486


In [82]:
# Check the shape of the item-customer interaction matrix

item_similarity_df.shape

(17, 17)

In [86]:
# Function to Recommend Similar Items Based on Item Similarity Scores:

# Retrieve Similarity Scores for the Target Item
def recommend_similar_items(item_id, item_similarity_df, num_recommendations=5):
    # Get the similarity scores for all items with the given item
    sim_scores = item_similarity_df.loc[item_id]

    #  # Sort items based on similarity scores in descending order
    sim_scores = sim_scores.sort_values(ascending=False)

    #  # Select indices of the most similar items, excluding the item itself
    recommended_items = sim_scores.index[1:num_recommendations+1].tolist()

    return recommended_items

### Testing the Recommendation System with Multiple Products.

In [111]:
## Generating Item Recommendations Based on Similarity for a Specified Product

# Specify the item ID and number of recommendations
item_id = 'Cleaners'  # Replace with an actual product subcategory from your dataset
num_recommendations = 8

# Generate recommendations
if item_id in item_similarity_df.index:
    recommended_items = recommend_similar_items(item_id, item_similarity_df, num_recommendations)
    print(f"Recommended items similar to '{item_id}': {recommended_items}")
else:
    print(f"'{item_id}' does not exist in the product subcategory list. Please input a valid product.")


Recommended items similar to 'Cleaners': ['Tires and Tubes', 'Bottles and Cages', 'Hydration Packs', 'Fenders', 'Mountain Bikes', 'Road Bikes', 'Bike Racks', 'Touring Bikes']


### Popularity based Approach

its a straightforward recommendation method where items are recommended based on their overall popularity rather than personalized preferences. This approach is particularly useful for new users (solving the "cold start" problem) or as a baseline for more complex recommendation models. Here’s how it works:

Steps for a Popularity-Based Recommendation Approach

Step 1. Create the Customer-Item Interaction Matrix: Create the Customer-Item Interaction Matrix
Use the customer purchase data to create a matrix where:
Rows represent each unique CustomerID.
Columns represent each ProductSubcategory.
Values represent the QuantityOrdered by each customer for each product category.
Replace any missing values with 0, indicating no interaction for that customer-product pair.


Step 2: Convert to a Sparse Matrix for Efficiency
Convert the interaction matrix to a Compressed Sparse Row (CSR) format using csr_matrix from scipy.sparse, which only stores non-zero values, saving memory and optimizing for large datasets.
Now, customer_item_matrix_sparse1 is ready for use in a recommendation engine, providing efficient storage and faster computations.

In [96]:
## Building and Optimizing the Customer-Item Interaction Matrix for Efficient Recommendations

# Create the customer-item interaction matrix
customer_item_matrix1 = df.pivot_table(index='CustomerID', columns='ProductSub', values='QuantityOrdered', fill_value=0)

# Convert to Sparse Matrix for Efficiency:
customer_item_matrix_sparse1 = csr_matrix(customer_item_matrix1.values)

# Display the Customer-Based Similarity Matrix
customer_item_matrix1

ProductSub,Bike Racks,Bike Stands,Bottles and Cages,Caps,Cleaners,Fenders,Gloves,Helmets,Hydration Packs,Jerseys,Mountain Bikes,Road Bikes,Shorts,Socks,Tires and Tubes,Touring Bikes,Vests
CustomerID,,,,,,,,,,,,,,,,,
11000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
11001,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
11002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
11003,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
11004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
29480,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
29481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
# Compute the cosine similarity matrix
customer_similarity_sparse1 = cosine_similarity(customer_item_matrix_sparse1)

# Convert the similarity matrix to a DataFrame for readability
customer_similarity_df1 = pd.DataFrame(customer_similarity_sparse1, 
                                       index=customer_item_matrix1.index, 
                                       columns=customer_item_matrix1.index)

# Display the Customer Similarity DataFrame
customer_similarity_df1

CustomerID,11000,11001,11002,11003,11004,11005,11006,11007,11008,11009,...,29474,29475,29476,29477,29478,29479,29480,29481,29482,29483
CustomerID,,,,,,,,,,,,,,,,,,,,,
11000,1.000000,0.617213,0.707107,0.547723,0.816497,0.707107,0.471405,0.912871,0.730297,0.816497,...,0.408248,0.408248,0.408248,0.471405,0.288675,0.408248,0.408248,0.408248,0.408248,0.408248
11001,0.617213,1.000000,0.436436,0.507093,0.566947,0.218218,0.436436,0.507093,0.507093,0.377964,...,0.377964,0.377964,0.377964,0.436436,0.267261,0.377964,0.566947,0.377964,0.377964,0.377964
11002,0.707107,0.436436,1.000000,0.516398,0.866025,0.666667,0.666667,0.774597,0.774597,0.866025,...,0.577350,0.577350,0.577350,0.666667,0.408248,0.577350,0.577350,0.577350,0.577350,0.577350
11003,0.547723,0.507093,0.516398,1.000000,0.447214,0.774597,0.774597,0.600000,0.800000,0.670820,...,0.447214,0.447214,0.447214,0.516398,0.632456,0.447214,0.670820,0.447214,0.447214,0.447214
11004,0.816497,0.566947,0.866025,0.447214,1.000000,0.577350,0.577350,0.894427,0.670820,0.750000,...,0.500000,0.500000,0.500000,0.577350,0.353553,0.500000,0.500000,0.500000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29479,0.408248,0.377964,0.577350,0.447214,0.500000,0.577350,0.577350,0.447214,0.447214,0.500000,...,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
29480,0.408248,0.566947,0.577350,0.670820,0.500000,0.288675,0.577350,0.447214,0.670820,0.500000,...,0.000000,0.000000,0.000000,0.866025,0.707107,0.000000,1.000000,0.000000,0.000000,0.000000
29481,0.408248,0.377964,0.577350,0.447214,0.500000,0.577350,0.577350,0.447214,0.447214,0.500000,...,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000


In [101]:
# Check the Dimensions of the Customer Similarity Matrix:
customer_similarity_sparse1.shape

(18484, 18484)

In [102]:
# Generate Item Recommendations Based on Customer Similarity

# Retrieve Index of the Target Customer
def recommend_items(customer_id, customer_similarity_sparse1, customer_item_matrix1, num_recommendations=5):
    # Get the index of the user
    customer_index = customer_item_matrix1.index.get_loc(customer_id)

    # Retrieve Similarity Scores for All Customers Related to the Target Custome
    sim_scores = customer_similarity_df1.iloc[customer_index]

    # Sort Customers by Similarity Scores in Descending Order
    sim_scores = sim_scores.sort_values(ascending=False)

    # Identify Indices of the Most Similar Customers (Excluding the Target Customer)
    sim_indices = sim_scores.index[1:]

    # Aggregate Items Interacted by Similar Customers
    similar_customers_items = customer_item_matrix1.loc[sim_indices].sum(axis=0)
    
    # Filter Out Items Already Interacted With by the Target Customer
    customer_items = customer_item_matrix1.loc[customer_id]
    similar_customers_items = similar_customers_items[customer_items == 0]

    # Select Top Recommended Items Based on Interaction Frequency
    recommended_items = similar_customers_items.sort_values(ascending=False).index[:num_recommendations].tolist()

    # Return the List of Recommended Items
    return recommended_items

### Testing the reccommendation engine

In [108]:
# Testing the Recommendation Engine for a Specific Custome

# Specify the customer ID and the number of recommendations to test
customer_id = 14503 # put one of the customer's id.
num_recommendations = 8


if customer_id in customer_similarity_df1:
   recommended_items = recommend_items(customer_id, customer_similarity_df1, customer_item_matrix1, num_recommendations)
   print(f"Recommended items for customer {customer_id}: {recommended_items}")

else:
    print(f'{Customer_id} does not exist, please enter a valid customer id')

Recommended items for customer 14503: ['Road Bikes', 'Bottles and Cages', 'Mountain Bikes', 'Jerseys', 'Touring Bikes', 'Caps', 'Fenders', 'Gloves']
